In [1]:
import pandas as pd
import numpy  as np
import time
from config import getClient

In [2]:
dataloc = getClient("OutFront")

In [3]:
keep = ['panelid', 'event_date_dt', 'hourinterval', 'populationcount']
df = pd.read_csv(dataloc+"0panels_original.csv",\
                 usecols=keep, dtype={"hourinterval":'int8',\
                                      "populationcount":'int32'})

# Rename the columns
cols = ["panel", "date", "hour", "population"]
df.columns = cols

# Convert their date format to a standard date
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d')

In [4]:
panels = df["panel"].unique()
panels = np.random.choice(panels, replace=False, size=500)

In [5]:
def writeError(panel, msg):
    rec = panel+","+msg+"\n"
    errors.write(rec)

In [6]:
def getFillRate(tmp):
    dates = tmp["date"]
    diff  = (dates.max() - dates.min()).days
    durations = diff*24
    numPoints = len(dates)
    return numPoints/durations

In [7]:
df = df.set_index("panel")
start = time.time()
dfList = []

output = open(dataloc+"1NEWfillRateFilter.csv", "w")
errors = open(dataloc+"errors.csv", "w")

count = 0

for panel in panels:
    count += 1
    if count %100 == 0: print(count)
    tmp = df.loc[panel]
    if tmp.shape[0] > 9000:
        rate = getFillRate(tmp)
        if rate > .92:
            tmp.sort_values(["date", "hour"], inplace=True)
            tmp = tmp.reset_index()
            dfList.append(tmp)
        else:
            writeError(panel, "Fill Rate too low")
    else:
        writeError(panel, "Duration too short")

final = pd.concat(dfList)
final.to_csv(dataloc+"1fillRateFilter.csv", index=False)

output.close()
errors.close()
end = time.time()
print("Completed after {:.0f} minutes".format((end-start)/60))

/home/tbrownex/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100
200
300
400
500
Completed after 2 minutes
